In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [146]:
import json

In [141]:
localities = pd.read_excel('data/localities_edit.xls', sheetname='Табл.36', converters={'bold':bool, 'italic':bool}).dropna(subset=['region']).reset_index()
#.dropna(subset=['total'])
del localities['index']

In [143]:
localities = localities.fillna(0)

In [104]:
from collections import defaultdict

In [162]:
localities.head(10)

,region,total,city,suburban,bold,italic
0,Республика Адыгея,451480.0,213542.0,237938.0,True,False
1,Городской округ - Город Майкоп,167350.0,144055.0,23295.0,True,True
2,г. Майкоп,144055.0,144055.0,0.0,False,False
3,Городской округ - Город Адыгейск,15161.0,12689.0,2472.0,True,True
4,г. Адыгейск,12689.0,12689.0,0.0,False,False
5,Гиагинский муниципальный район,31185.0,0.0,31185.0,True,True
6,Айрюмовское сельское поселение,3100.0,0.0,3100.0,False,False
7,Гиагинское сельское поселение,15376.0,0.0,15376.0,False,False
8,Дондуковское сельское поселение,6753.0,0.0,6753.0,False,False
9,Келермесское сельское поселение,3061.0,0.0,3061.0,False,False


In [140]:
selo_types = ['р-н', 'район', 'г.', 'город', 'п.г.т.', 'пгт.' 'посёлок городского типа', 'р.п.', 'рабочий посёлок', 
              'к.п.', 'курортный посёлок', 'к.', 'кишлак', 'пс', 'поселковый совет', 'поссовет', 'сс', 'сельсовет', 
              'сельский совет', 'смн', 'сомон', 'вл.', 'волость', 'д.п.', 'дачный поселковый совет', 'п.', 
              'посёлок сельского типа', 'н.п.', 'населённый пункт', 'п. ст.', 'посёлок при станции', 'ж/д ст.', 
              'железнодорожная станция', 'с.', 'село', 'м.', 'местечко', 'д.', 'деревня', 'сл.', 'слобода', 'ст.',
              'станция', 'ст-ца', 'станица', 'х.', 'хутор', 'у.', 'улус', 'рзд.', 'разъезд', 'клх', 'колхоз', 'свх',
              'совхоз', 'зим.', 'зимовье']

In [235]:
localities.loc[8155]#.region.lower().split('сельское поселение')[0].strip()

region      Дмитриевское сельское поселение
total                                  3571
city                                      0
suburban                               3571
bold                                  False
italic                                False
Name: 8155, dtype: object

In [259]:
from collections import OrderedDict

In [260]:
cnt = 0
loc_regions = defaultdict(list)
loc_data = []
locality = False
prev_locality = {'region': 0, 'name':0, 'total':0, 'city':0, 
                                     'suburban':0, 'type':0}
for i, row in localities.iterrows():
    name = ''
    agl_type = ''
    if row.region == 'г. Москва' or row.region == 'г. Санкт-Петербург' or row.region =='г. Севастополь':
        name = row.region.lower().split('г. ')[1]
        agl_type = 'город'
        cnt += row.total
        loc_regions[cur_region] += [OrderedDict({'region': row.region, 'name':name.lower(), 'total':row.total, 'city':row.city, 
                                     'suburban':row.suburban, 'type':agl_type})]
    if row.bold:
        if not row.italic:
            cur_region = row.region
            loc_regions[cur_region] = []
        else:
            locality = True
    else:
        if locality:
            if 'внутригородские районы' in row.region.lower():
                locality = False
                continue
            name = ''
            agl_type = ''
            if len(row.region.lower().split('сельское поселение')) == 2:
                for elem in row.region.lower().split('сельское поселение'):
                    if len(elem) > 1:
                        name = elem.strip()
#                         print(name, i)
                for st in selo_types:
                    if st + ' ' in name or ' ' + st in name:
                        name = name.replace(st, '').strip()
                agl_type = 'сельское поселение'
            elif row.region.startswith('г. '):
                name = row.region.split('г. ')[1].strip()
                agl_type = 'город'
                if prev_locality['total'] == row.total:
                    print(prev_locality)
                    loc_regions[cur_region].remove(prev_locality)
            elif row.region.startswith('пгт.'):
                name = row.region.split('пгт.')[1].strip()
                if ' (' in name:
                    name = name.split(' (')[0]
                agl_type = 'пгт'
                if prev_locality['total'] == row.total:
                    
                    loc_regions[cur_region].remove(prev_locality)
            if name:
                prev_locality = OrderedDict({'region': cur_region, 'name':name.lower(), 'total':row.total, 'city':row.city, 
                                     'suburban':row.suburban, 'type':agl_type})
                loc_regions[cur_region] += [prev_locality]
                cnt += row.total if row.total else 0
            
            
            
#             print(cur_region)
            

In [232]:
pd.read_json([{'cnt': 2355.0,
               'name': 'зебляковское',
               'type': 'сельское поселение'},
              {'cnt': 2212.0,
               'name': 'ивановское',
               'type': 'сельское поселение'}], orient='records')

TypeError: Expected String or Unicode

In [230]:
cnt

139345507.0

In [261]:
loc_regions_df = []
for v in loc_regions.values():
    for v1 in v:
        loc_regions_df += [v1]

In [265]:
loc_regions

[]

In [266]:
loc_regions_df = pd.DataFrame.from_dict(loc_regions_df)

In [271]:
loc_regions_df.

,suburban,type,name,region,city,total
0,0.0,город,элиста,Республика Калмыкия,104005.0,104005.0
1,0.0,город,городовиковск,Республика Калмыкия,8799.0,8799.0
2,0.0,город,лагань,Республика Калмыкия,13170.0,13170.0


(106908, 5)

In [282]:
loc_regions_df

,suburban,type,name,region,city,total
0,0.0,город,элиста,Республика Калмыкия,104005.0,104005.0
1,0.0,город,городовиковск,Республика Калмыкия,8799.0,8799.0
2,0.0,город,лагань,Республика Калмыкия,13170.0,13170.0
3,0.0,город,краснодар,Краснодарский край,853848.0,853848.0
4,0.0,город,анапа,Краснодарский край,73410.0,73410.0
5,0.0,город,армавир,Краснодарский край,191007.0,191007.0
6,0.0,город,геленджик,Краснодарский край,72030.0,72030.0
7,0.0,город,горячий ключ,Краснодарский край,35805.0,35805.0
8,0.0,город,новороссийск,Краснодарский край,266977.0,266977.0
9,0.0,город,сочи,Краснодарский край,401291.0,401291.0


In [276]:
merged = loc_regions_df.merge(coords, left_on=['region', 'name'], right_on=['region', 'locality'], how='left')

In [281]:
merged[merged['type'] == 'город'].lat.isnull().sum()

196

In [278]:
merged.lat.isnull().sum()

16087

In [257]:
with open('localities_dict.json', 'w') as f:
    json.dump(loc_regions, f, indent=4)

In [269]:
coords.rajon = coords.rajon.apply(lambda x: x.lower())
coords.locality = coords.locality.apply(lambda x: x.lower())

In [88]:
coords = pd.read_csv('data/ru-list.csv', sep=';', header=None)
coords.columns =  ['region', 'rajon', 'locality', 'lon', 'lat']

In [89]:
coord_region = pd.DataFrame(coords.region.unique())
coord_region.columns = ['name']

In [272]:
coords.head()

,region,rajon,locality,lon,lat
0,Республика Адыгея,гиагинский,владимировское,40.200,44.750
1,Республика Адыгея,гиагинский,вольно-веселый,40.266,44.916
2,Республика Адыгея,гиагинский,гавриловский,40.183,44.966
3,Республика Адыгея,гиагинский,георгиевское,40.416,44.766
4,Республика Адыгея,гиагинский,гиагинская,40.660,44.866


In [90]:
pd.options.display.max_rows = 100

In [93]:
def oblast(x):
    if x.endswith('ая'):
        return x + ' область'
    else:
        return x
def ao(x):
    if 'ий АО' in x:
        x = x.replace('АО', 'автономный округ')
    elif 'ая АО' in x:
        x = x.replace('АО', 'автономная область')
    return x

coord_region['name_new'] = coord_region.name.apply(oblast)
coord_region['name_new'] = coord_region.name_new.apply(ao)

In [99]:
map_region = dict((v[0], v[1]) for v in coord_region.values)

In [101]:
coords.region = coords.region.apply(lambda x: map_region[x])

In [83]:
for elem2 in loc_regions:
    if not (elem2 in coord_region.name):
        print(elem2)

Республика Адыгея
Республика Алтай
Республика Башкортостан
Республика Бурятия
Республика Дагестан
Республика Ингушетия
Кабардино-Балкарская Республика
Республика Калмыкия
Карачаево-Черкесская Республика
Республика Карелия
Республика Коми
Республика Крым
Республика Марий Эл
Республика Мордовия
Республика Саха (Якутия)
Республика Северная Осетия-Алания
Республика Татарстан
Республика Тыва
Удмуртская Республика
Республика Хакасия
Чеченская Республика
Чувашская Республика
Алтайский край
Забайкальский край
Камчатский край
Краснодарский край
Красноярский край
Пермский край
Коми-Пермяцкий округ
Приморский край
Ставропольский край
Хабаровский край
Амурская область
Архангельская область
Ненецкий автономный округ
Астраханская область
Белгородская область
Брянская область
Владимирская область
Волгоградская область
Вологодская область
Воронежская область
Ивановская область
Иркутская область
Калининградская область
Калужская область
Кемеровская область
Кировская область
Костромская область
Курганск

In [102]:
np.intersect1d(coords.region.unique(), np.array(loc_regions)).shape

(81,)

In [125]:
data = []
for elem in s.split(';\n'):
    elem = elem.split(' — ')
    data += [elem[0], elem[1]]

In [208]:
test = pd.read_csv('hostpitals_final.csv')
test = test[['lat', 'lon']]
test.columns = ['lat','lng']
test.to_json('test.json', orient='records')

In [212]:
test1 = pd.read_csv('agloms_final.csv', index_col=0)